# Look-through Valuation and Reporting

## Overview

In this Notebook, we present some of the look-through valuation capabilities available in LUSID across a multi-level security and fund structure. We look at these in the context of several securitised portfolios as well as an Equity Index Future. 

Our securitised portfolio hiearchy is constructed of three levels including a 'Global Equity' portfolio at the top level, and three sector specific securitised portfolios at the second level. 

The three sector specific portfolios focus on financials, energy, and technology. Each contains direct equity holdings in individual stocks. In addition:
1. The 'Global Equity Technology' portfolio holds an additional position in an Equity Index Future (the NYSE FANG+).

2. The 'Global Equity Financials' portfolio holds a position in the 'US Equity Financials' Fund

In the proceeding sections of this Notebook, we'll load all of the requisite data required to model these holdings while performing valuations for January 7th, 2022. 

Importantly, we'll walk through how we can drill down to the underlying positions and index constituents of our holdings in order to give us a clear view of our exposures. 

At a high level, the structure of our example looks as follows:

![Init](img/LookthroughMultiExample.PNG)

## Setup

We first initialize our various Python libraries, objects, and datasets required to construct our examples:

In [ ]:
# Import common libraries
import os
import pandas as pd
import logging
import pytz
import json
import random
from datetime import datetime, timezone
from IPython.core.display import HTML
logging.basicConfig(level = logging.INFO)

# Import LUSID libraries
import lusid as lu
import lusid.models as lm

import lusidjam
import lusid.extensions as le
from finbourne_sdk_utils.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from finbourne_sdk_utils.lpt.lpt import to_date
from finbourne_sdk_utils import cocoon as cocoon

# Set pandas display options
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format

# Authenticate to SDK
# Run the Notebook in Jupyterhub for your LUSID domain and authenticate automatically
secrets_path = os.getenv("FBN_SECRETS_PATH")
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
config_loaders=[
    le.ArgsConfigurationLoader(access_token = lusidjam.RefreshingToken(), app_name = "LusidJupyterNotebook"),
    le.EnvironmentVariablesConfigurationLoader(),
    le.SecretsFileConfigurationLoader(secrets_path)]
api_factory = le.SyncApiClientFactory(config_loaders=config_loaders)

# Confirm success
api_client = api_factory.build(lu.ApplicationMetadataApi)
api_url = api_client.api_client.configuration._base_path.replace("api","")

print ('LUSID Environment :', api_url + "docs")
display(pd.DataFrame(api_client.get_lusid_versions().to_dict()))

In [ ]:
portfolio_api = api_factory.build(lu.PortfoliosApi)
properties_api = api_factory.build(lu.PropertyDefinitionsApi)
transaction_portfolio_api = api_factory.build(lu.TransactionPortfoliosApi)
reference_portfolio_api = api_factory.build(lu.ReferencePortfolioApi)
instruments_api = api_factory.build(lu.InstrumentsApi)
quotes_api = api_factory.build(lu.QuotesApi)
configuration_recipe_api = api_factory.build(lu.ConfigurationRecipeApi)
relationship_definitions_api = api_factory.build(lu.RelationshipDefinitionsApi)
relationships_api = api_factory.build(lu.RelationshipsApi)
aggregration_api = api_factory.build(lu.AggregationApi)

In [ ]:
# Set Global Scope
global_scope = "drill-through"
effective_date = datetime(2022, 1, 7, tzinfo=timezone.utc)

# Transaction Portfolios
transaction_portfolios = ["GlobalEquity", "GlobalEquityEnergy", "USEquityFinancials", "GlobalEquityFinancials", "GlobalEquityTechnology"]
fund_portfolios = transaction_portfolios[:4]

# Reference Portfolios
reference_portfolios = ["NYSEFANGPIndex"]

# Recipes
direct_holdings_recipe = "DirectHoldingsValuationRecipe"
fund_lookthrough_recipe = "FundLookthroughValuationRecipe"
full_lookthrough_recipe = "FullLookthroughValuationRecipe"

# Load Requisite Data
transaction_data = pd.read_excel("data/look-through/lookthrough_data.xlsx", sheet_name="transactions")
index_data = pd.read_excel("data/look-through/lookthrough_data.xlsx", sheet_name="index_weights")
price_data = pd.read_excel("data/look-through/lookthrough_data.xlsx", sheet_name="market_prices")
instrument_data = pd.read_excel("data/look-through/lookthrough_data.xlsx", sheet_name="instruments")

## Delete Existing Data

In [ ]:
for portfolio in transaction_portfolios + reference_portfolios:
    try:
        portfolio_api.delete_portfolio(global_scope, portfolio)
        print("Deleted portfolio: " + portfolio)
    except lu.ApiException as e:
        print(f'{portfolio}: {json.loads(e.body)["title"]}')  

## Load Data

The majority of our data will be loaded from 'lookthrough_data.xlsx'. This includes our transactions, the majority of our instruments, market quotes, and the index constituent data of our Equity Index Future.

### Portfolios 

We first start by constructing our portfolios. This includes: 

1. A single top level LUSID Transacion Portfolio representing the Global Equity portfolio.
1. Three Transaction Portfolios covering the energy, financials, and technology sectors. These will ultimately be securitised and referred to within a set of instrument definitions.
3. A Reference Portfolios containing an equity index (NYSE FANG+) which will be used to model the underlying constituents of a corresponding Equity Index Future.

##### Transaction Portfolios

In [ ]:
# Create our Transaction Portfolios
def load_txn_portfolio(portfolio_code):
    transaction_portfolio_api.create_portfolio(
        scope=global_scope,
        create_transaction_portfolio_request=lm.CreateTransactionPortfolioRequest(
            display_name=f"{portfolio_code} Portfolio",
            code=portfolio_code,
            base_currency="USD",
            created=effective_date.replace(year=effective_date.year - 1),
            instrument_scopes=[global_scope]
        ),
    )
    print("Portfolio: " + portfolio_code + " loaded!")

for portfolio in transaction_portfolios:
    load_txn_portfolio(portfolio)

#### Reference Portfolio

In [ ]:
# Create a Reference Portfolio for our index future
def load_ref_portfolio(portfolio_code):
    response = reference_portfolio_api.create_reference_portfolio(
        scope=global_scope,
        create_reference_portfolio_request=lm.CreateReferencePortfolioRequest(
            display_name=f"{portfolio} Portfolio",
            base_currency="USD",
            code=portfolio, 
            created=effective_date.replace(year=effective_date.year - 1),
            instrument_scopes=[global_scope]
        ),
    )

for portfolio in reference_portfolios:
    load_ref_portfolio(portfolio)

### Instruments

#### Instrument Properties
Next, we'll define the instrument properties for which we'll aggregate on. These include:

- Sector
- Industry
- Region

In [ ]:
# Create instrument properties for use in a valuation aggregation
properties = [
    ("Sector", "string"),
    ("Industry", "string"),
    ("Region", "string") 
]

for property_code, dtype in properties:
    try:
        properties_api.create_property_definition(
            create_property_definition_request=lm.CreatePropertyDefinitionRequest(
                domain="Instrument",
                scope=global_scope,
                code=property_code,
                display_name=property_code,
                data_type_id=lm.ResourceId(code=dtype, scope="system"),
            )
        )
    except lu.ApiException as e:
        print(json.loads(e.body)["title"])

#### Create a Lookthrough Instrument
For our first instrument, we'll be securitising the NYSE FANG+ Equity Index using a `SimpleInstrument` instrument type. We must do this in order to refer to the Index within a Future instrument. Futures cannot link to an Index in LUSID directly, rather, they must refer to a securitized version of it.

Given that our Index is constructed by specyfing a set of constituents and corresponding weights, we'll model it using a Reference Portfolio. For additional details on Reference Portfolios, please see the following [KB article](https://support.lu.com/knowledgebase/article/KA-01852/en-us). 

Key to our `SimpleInstrument` construction is the field `look_through_portfolio_id`. This is where we specify our Reference Portfolio code. The securitized Reference Portfolio construction is thus as follows:

In [ ]:
equity = lm.SimpleInstrument(
    instrument_type="SimpleInstrument",
    dom_ccy="USD",
    asset_class="Equities",
    simple_instrument_type="EquityIndex"
)

equity_definition = lm.InstrumentDefinition(
    name="ref_NYSEFANGInst",
    identifiers={ "ClientInternal": lm.InstrumentIdValue(value="ref_NYSEFANGInst") },
    definition=equity,
    look_through_portfolio_id=lm.ResourceId(scope=global_scope, code="NYSEFANGPIndex")
)

upsert_request = {"ref_NYSEFANGInst": equity_definition}
upsert_response = api_factory.build(lu.api.InstrumentsApi).upsert_instruments(
    scope=global_scope,
    request_body=upsert_request
)

equity_lookthroughinst_luid = upsert_response.values["ref_NYSEFANGInst"].lusid_instrument_id
print(f"LUID for newly created securitised instrument: {equity_lookthroughinst_luid}")

#### Equities
Next, we load our publicly listed equity instruments as type 'Equity'.

In [ ]:
instrument_data

In [ ]:
# Load publicly listed equities
def load_equity(data):
          
    client_internal = "Instrument/default/ClientInternal"
    
    equity = lm.Equity(
        instrument_type="Equity",
        dom_ccy=data["currency"],
    )

    equity_definition = lm.InstrumentDefinition(
        name=data["name"],
        identifiers={"ClientInternal": lm.InstrumentIdValue(value=data["client_internal"]),
                     "Ticker": lm.InstrumentIdValue(value=data["ticker"])
                    },
        definition=equity,
        properties=[
                lm.ModelProperty(
                key=f"Instrument/{global_scope}/Sector",
                value=lm.PropertyValue(label_value=row['sector']),
            ),
                lm.ModelProperty(
                key=f"Instrument/{global_scope}/Industry",
                value=lm.PropertyValue(label_value=row['industry']),
            ),
                lm.ModelProperty(
                key=f"Instrument/{global_scope}/Region",
                value=lm.PropertyValue(label_value=row['region']),
            ),
        ],        
    )

    # upsert the instrument
    upsert_request = {client_internal: equity_definition}
    upsert_response = instruments_api.upsert_instruments(scope=global_scope, request_body=upsert_request)
    equity_luid = upsert_response.values[client_internal].lusid_instrument_id

for index, row in instrument_data.iterrows():
    
    if not isinstance(row["lookthrough_id"], float):
        continue
        
    load_equity(row)

print ("Instruments Upserted")

#### Equity Index Future
Our equity index future will be modelled as a LUSID 'Future' instrument. Importantly, it will refer to the ref_NYSEFANGInst 'SimpleInstrument' via a 'ReferenceInstrument' underlying reference. We do this by passing in the ref_NYSEFANGInst's corresponding LusidInstrumentId.

In [ ]:
# Function definition to create a future
def create_futures_contract(
        dom_ccy,
        contract_code,
        contract_month,
        contract_size,
        convention,
        country_id,
        fut_name,
        exchange_code,
        exchange_name,
        ticker_step,
        unit_value,
        ref_spot_price,
        start_date,
        maturity_date,
        fut_identifier,
):
    ctc = lm.FuturesContractDetails(
        dom_ccy=dom_ccy,
        contract_code=contract_code,
        contract_month=contract_month,
        contract_size=contract_size,
        convention=convention,
        country=country_id,
        description=fut_name,
        exchange_code=exchange_code,
        exchange_name=exchange_name,
        ticker_step=ticker_step,
        unit_value=unit_value,
    )
     
    futuredef = lm.Future(
        start_date=start_date,
        maturity_date=maturity_date,
        identifiers={},
        contract_details=ctc,
        contracts=1,
        ref_spot_price=ref_spot_price,
        underlying=lm.ReferenceInstrument(
            instrument_type="ReferenceInstrument",
            instrument_id=equity_lookthroughinst_luid, #LUID of look-through SimpleInstrument defined in 3.2.2
            instrument_id_type="LusidInstrumentId",
            scope=global_scope
        ),
        instrument_type="Future",
    )

    # persist the instrument
    futureDefinition = lm.InstrumentDefinition(
        name=fut_name,
        identifiers={"ClientInternal": lm.InstrumentIdValue(value=fut_identifier)},     
        definition=futuredef,
        properties=[
            lm.ModelProperty(
                key=f"Instrument/{global_scope}/Sector",
                value=lm.PropertyValue(label_value="Technology"),
            ),
            lm.ModelProperty(
                key=f"Instrument/{global_scope}/Industry",
                value=lm.PropertyValue(label_value="Multi"),
            ),
            lm.ModelProperty(
                key=f"Instrument/{global_scope}/Region",
                value=lm.PropertyValue(label_value="Global"),
            ),
        ],             
    )

    batchUpsertRequest = {fut_identifier: futureDefinition}
    upsertResponse = instruments_api.upsert_instruments(scope=global_scope, request_body=batchUpsertRequest)
    fut_luid = upsertResponse.values[fut_identifier].lusid_instrument_id
    print(f"LUID for newly created Future instrument: {fut_luid}")

In [ ]:
#NYSE FANG+ Future Definition
create_futures_contract(
    dom_ccy="USD",
    contract_code="FNG",
    contract_month="M",
    contract_size=5,
    convention="ActualActual",
    country_id="US",
    fut_name="Micro NYSE FANG+ Index Future Mar22",
    exchange_code="ICE",
    exchange_name="ICE Exchange",
    ticker_step=0.2,
    unit_value=1,
    ref_spot_price=None,
    start_date = datetime(2021, 6, 21, tzinfo=pytz.utc),
    maturity_date = datetime(2022, 6, 17, tzinfo=pytz.utc),
    fut_identifier="fut_NYSEFANGP001"
)

#### Funds (Securitised Transaction Portfolios)
Finally, we must securitise our industry specific Transaction Portfolios by creating a set of instruments which refer to them via the 'look_through_portfolio_id' field.

In [ ]:
# Securitise our transaction portfolios
for index, row in instrument_data.iterrows():

    if isinstance(row["lookthrough_id"], float):
        continue
        
    simple_instrument = lm.SimpleInstrument(
        instrument_type="SimpleInstrument",
        dom_ccy=row["currency"],
        asset_class="Equities",
        simple_instrument_type="Fund"
    )

    response = instruments_api.upsert_instruments(
        scope=global_scope,
        request_body={
            "upsert_instrument": lm.InstrumentDefinition(
                name=row["name"],
                definition=simple_instrument,
                identifiers={
                    "ClientInternal": lm.InstrumentIdValue(value=row["client_internal"])
                },
                look_through_portfolio_id=lm.ResourceId(
                    scope=global_scope, code=row["lookthrough_id"]
                ),
                properties=[
                        lm.ModelProperty(
                        key=f"Instrument/{global_scope}/Sector",
                        value=lm.PropertyValue(label_value=row['sector']),
                    ),
                        lm.ModelProperty(
                        key=f"Instrument/{global_scope}/Industry",
                        value=lm.PropertyValue(label_value=row['industry']),
                    ),
                        lm.ModelProperty(
                        key=f"Instrument/{global_scope}/Region",
                        value=lm.PropertyValue(label_value=row['region']),
                    ),
                ],
            )
        }
    )
    
print ("Securitised Funds Upserted!")   

### 3.3 Index Constituents

Next, we want to load the constituent data of the NYSE FANG+ Index

In [ ]:
index_data

We specify quarterly rebalancing of the index via the `period_type` and `weight_type` fields. For additional details on Reference Portfolio construction, please see the following [KB article](https://support.lu.com/knowledgebase/article/KA-01852/en-us).

In [ ]:
# Initialise a list to hold our constituents
constituents = [
        lm.ReferencePortfolioConstituentRequest(
        instrument_identifiers={
            "Instrument/default/Ticker": row["ticker"]
        },
        weight=row["weight"],
        currency="USD",
        ) for _, row in index_data.iterrows()]

# Create our request to add our constituents
constituents_request = lm.UpsertReferencePortfolioConstituentsRequest(
    effective_from=effective_date.isoformat(),
    weight_type="Periodical",
    period_type="Quarterly",
    period_count=1,
    constituents=constituents,
)

# Call LUSID to upsert our constituents into our reference portfolio
response = api_factory.build(
    lu.api.ReferencePortfolioApi
).upsert_reference_portfolio_constituents(
     scope=global_scope,
     code="NYSEFANGPIndex",
    upsert_reference_portfolio_constituents_request=constituents_request,
)

print(f"Constituents Upserted!")

### Transactions
To construct our holdings, we load in a set of transaction data across our four portfolios

In [ ]:
# Load Transactions
for index, row in transaction_data.iterrows():

    primary_instrument_identifier = { "Instrument/default/ClientInternal": row["client_internal"] }
    
    if isinstance(row["client_internal"], float):
        primary_instrument_identifier = { "Instrument/default/Currency": row["cash_id"] }

    upsert_transactions = transaction_portfolio_api.upsert_transactions(
        scope=global_scope,
        code=row['portfolio'],
        transaction_request=[
            lm.TransactionRequest(
                transaction_id=row["txn_id"],
                type=row["txn_type"],
                instrument_identifiers=primary_instrument_identifier,
                transaction_date=row["trade_date"],
                settlement_date=row["settle_date"],
                units=row["quantity"],
                transaction_price=lm.TransactionPrice(
                    price=row["txn_price"], type="Price"
                ),
                total_consideration=lm.CurrencyAndAmount(
                    amount=row["total_consideration"], currency=row["currency"]
                ),
            )
        ],
    )

### Quotes
We now load in the relavent market prices for our valuation date of Jan 7th, 2022

In [ ]:
# Display Instrument Prices
price_data

In [ ]:
instrument_quotes = {
    f'{price["id"]}-{price["date"]}': lm.UpsertQuoteRequest(
            quote_id=lm.QuoteId(
                quote_series_id=lm.QuoteSeriesId(
                    provider="Lusid",
                    instrument_id=price["id"],
                    instrument_id_type=price["id_type"],
                    quote_type="Price",
                    field="mid",
                ),
                effective_at=price["date"],
            ),
            metric_value=lm.MetricValue(value=price["price"], unit=price["currency"]),        
    )
    for row, price in price_data.iterrows()
}

# Upsert the quotes into LUSID
response = quotes_api.upsert_quotes(
    scope=global_scope, request_body=instrument_quotes
)

## Run Valuations

Now that the requisite data is in place to model our portfolios and holdings, we want to start running valuations. In order to do this, we'll create three recipes:

- `DirectHoldingsValuationRecipe` - The top level valuation recipe which will value all directly held positions with no look-through
- `FundLookthroughValuationRecipe` - The fund level valuation recipe will look through all securitised portfolios.
- `FullLookthroughValuationRecipe` - The full look-through valuation recipe will expand all securitised portfolios and index futures.

Importantly, we have two model rules that control look-through for `SimpleInstrument` and `Future` instrument types. The first model rule applies to instruments of type `SimpleInstrument`, and the specification of the `IndexModelOptions` under the `model_options` field tells LUSID to expand those instruments into their underlying holdings, and how to scale the underlying portfolio PVs.

In our second model rule, we again specify an `IndexModelOptions` object to tell LUSID how we want to expand the Index Future during valuation. If we do not specify this, the Index Future will not expand to its underlying constituents.

Note that our recipe below depends on two types of LUSID licenses. First, we need a license for each instrument type we're working with: Equity, `SimpleInstrument`, and `Future`. Second, we need licenses for the pricing models we're using: in this case, `SimpleStatic`. For additional details on Recipes, please see the following [KB article](https://support.lu.com/knowledgebase/article/KA-01895/en-us).

### 4.1 Create Recipes

In [ ]:
# Create three recipes for varying levels of look-through
def UpsertRecipe(recipe_code, drill_through_simple_instruments, drill_through_futures):
    
    model_rules=[]
        
    if drill_through_simple_instruments:
        model_rules.append(lm.VendorModelRule(
                supplier="Lusid",
                model_name="SimpleStatic",
                instrument_type="SimpleInstrument",
                model_options=lm.IndexModelOptions(portfolio_scaling="AbsoluteSum", model_options_type="IndexModelOptions"),
                )
        )        

    if drill_through_futures:
        model_rules.append(lm.VendorModelRule(
                supplier="Lusid",
                model_name="SimpleStatic",
                instrument_type="Future",
                model_options=lm.IndexModelOptions(portfolio_scaling="AbsoluteSum", model_options_type="IndexModelOptions"),
            )
        )
    
    configuration_recipe = lm.ConfigurationRecipe(
        scope=global_scope,
        code=recipe_code,
        market=lm.MarketContext(
            market_rules=[
                lm.MarketDataKeyRule(
                    key="Equity.ClientInternal.*",
                    supplier="Lusid",
                    data_scope=global_scope,
                    quote_type="Price",
                    field="mid",
                    quote_interval="5D",
                )
            ],
            options=lm.MarketOptions(
                default_supplier="Lusid",
                default_instrument_code_type="ClientInternal",
                default_scope=global_scope,
                attempt_to_infer_missing_fx=True             
            ),
        ),
        pricing=lm.PricingContext(
            model_rules=model_rules
        ),    
    )

    upsert_configuration_recipe_response = (
        configuration_recipe_api.upsert_configuration_recipe(
            upsert_recipe_request=lm.UpsertRecipeRequest(
                configuration_recipe=configuration_recipe
            )
        )
    )
    
UpsertRecipe(direct_holdings_recipe, False, False)
UpsertRecipe(fund_lookthrough_recipe, True, False)
UpsertRecipe(full_lookthrough_recipe, True, True)

### Create Valuation Function

We'll create a valuation function that allows us to aggregate on an instrument property. In our example, this will either be by industry, sector, or region.

In [ ]:
# Create a valuation function allowing users to aggregate based on different instrument properies
# as well as by specifying the look-through recipe
def get_daily_val(date, portfolio_code, aggregate_field, recipe_code):
    
    group_by = []
    metricsList = []
    columnsToRename = {}
    
    if aggregate_field:
        metricsList.extend([
            lm.AggregateSpec(key=f"Instrument/{global_scope}/{aggregate_field}", op="Value"),
            lm.AggregateSpec(key=f"Valuation/PvInReportCcy", op="Sum"),        
            lm.AggregateSpec(key=f"Valuation/ExposureInReportCcy", op="Sum"),             
            lm.AggregateSpec(key=f"Holding/default/Units", op="Value"),              
        ])
        group_by.append(f"Instrument/{global_scope}/{aggregate_field}")
        columnsToRename={
            f"Sum(Valuation/PvInReportCcy)": "PV (Reporting Ccy)",
            f"Sum(Valuation/ExposureInReportCcy)": "Exposure (Reporting Ccy)"
        }
        
    else:
        metricsList.extend([
            lm.AggregateSpec(key=f"Instrument/default/Name", op="Value"),
            lm.AggregateSpec(key=f"Instrument/{global_scope}/Sector", op="Value"), 
            lm.AggregateSpec(key=f"Instrument/{global_scope}/Industry", op="Value"),
            lm.AggregateSpec(key=f"Instrument/{global_scope}/Region", op="Value"),
            lm.AggregateSpec(key=f"Valuation/PvInReportCcy", op="Value"),
            lm.AggregateSpec(key=f"Valuation/ExposureInReportCcy", op="Value"),
            lm.AggregateSpec(key=f"Holding/default/FundLineage", op="Value"),
            lm.AggregateSpec(key=f"Holding/default/Units", op="Value"),
        ])
        columnsToRename={
            f"Valuation/PvInReportCcy": "PV (Reporting Ccy)",
            f"Valuation/ExposureInReportCcy": "Exposure (Reporting Ccy)",
            f"Holding/default/FundLineage": "Fund Lineage",
        }
          
    # Build and run valuation request
    valuation_request = lm.ValuationRequest(
        recipe_id=lm.ResourceId(scope=global_scope, code=recipe_code),
        metrics=metricsList,
        group_by=group_by,
        portfolio_entity_ids=[
            lm.PortfolioEntityId(scope=global_scope, code=portfolio_code)
        ],
        valuation_schedule=lm.ValuationSchedule(effective_at=date.isoformat()),
    )

    val_data = aggregration_api.get_valuation(valuation_request=valuation_request).data
    vals_df = pd.DataFrame(val_data)
    
    if aggregate_field:
        columnsToRename[f"Instrument/{global_scope}/{aggregate_field}"] = f"{aggregate_field}"
    else:
        columnsToRename[f"Instrument/default/Name"] = "Instrument Name"
        columnsToRename[f"Instrument/{global_scope}/Sector"] = "Sector"
        columnsToRename[f"Instrument/{global_scope}/Industry"] = "Industry"
        columnsToRename[f"Instrument/{global_scope}/Region"] = "Region"

    vals_df.rename(
        columns=columnsToRename,
        inplace=True,
    )

    vals_df.drop('Holding/default/Units', axis=1, inplace=True)

    return vals_df

### Top Level Valuation

We first look at a top-level valuation report unaggregated

In [ ]:
get_daily_val(effective_date, "GlobalEquity", "", direct_holdings_recipe)

We then decide to aggregate on Sector

In [ ]:
get_daily_val(effective_date, "GlobalEquity", "Sector", direct_holdings_recipe)

### Fund Level Look-through Valuation

Next, we want to value our portfolio by drilling through to each securitised portfolio's underlying constituent position. We do this by using the 'FundLookthroughValuationRecipe' recipe.

In [ ]:
get_daily_val(effective_date, "GlobalEquity", "", fund_lookthrough_recipe)

We then aggregate by Industry

In [ ]:
get_daily_val(effective_date, "GlobalEquity", "Industry", fund_lookthrough_recipe)

It's important to note that LUSID performs look-through to the lowest level of granularity possible. In our example, we have a two level fund hierachy. If however this hierachy was three or more levels, LUSID would drill down to the 'leaf level' positions of this hierachy (i.e. where positions have no further lookthrough data constructed).

### Full Look-through Valuation

Lastly, we want to value our portfolio by drilling down at both the securitised fund level, as well as at the Index Future constituent level.

In [ ]:
get_daily_val(effective_date, "GlobalEquity", "", full_lookthrough_recipe)

We'll now aggregate on region. You'll notice the two Chinese based equities held in the Index Future (Baidu and Alibaba) add to the fund's China exposure.

In [ ]:
get_daily_val(effective_date, "GlobalEquity", "Region", full_lookthrough_recipe)

## View in the Web UI

In [ ]:
display(HTML(f'<a href="{api_url}app/dashboard/valuations?scope={global_scope}&code=GlobalEquity&entityType=Portfolio&recipeScope={global_scope}&recipeCode={direct_holdings_recipe}&effectiveDate={effective_date.isoformat()}" target="_blank">See direct holdings</a>'))

display(HTML(f'<a href="{api_url}app/dashboard/valuations?scope={global_scope}&code=GlobalEquity&entityType=Portfolio&recipeScope={global_scope}&recipeCode={fund_lookthrough_recipe}&effectiveDate={effective_date.isoformat()}" target="_blank">Fund drill-through holdings</a>'))

display(HTML(f'<a href="{api_url}app/dashboard/valuations?scope={global_scope}&code=GlobalEquity&entityType=Portfolio&recipeScope={global_scope}&recipeCode={full_lookthrough_recipe}&effectiveDate={effective_date.isoformat()}" target="_blank">Full drill-through holdings</a>'))